In [ ]:
!pip install --upgrade --quiet langchain langchain-community langchain-huggingface chromadb gradio sentence-transformers


In [ ]:
import os
import ast
import re
from langchain_community.utilities import SQLDatabase
from langchain_community.vectorstores import Chroma
from langchain_community.agent_toolkits import create_sql_agent
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import ChatPromptTemplate, FewShotPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceHub
from langchain.agents.agent_toolkits import create_retriever_tool
import gradio as gr

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Đường dẫn đến database
sql_path = '/content/drive/MyDrive/chat/products.db'

In [ ]:
# Khởi tạo SQL Database
db = SQLDatabase.from_uri(f"sqlite:////{sql_path}")
print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")

In [ ]:
# Kiểm tra dữ liệu
print(db.run("select * from products limit 5"))

In [ ]:
# Kiểm tra schema của database
def get_schema_info():
    schema_info = {}
    for table in db.get_usable_table_names():
        columns = db.run(f"PRAGMA table_info({table})")
        schema_info[table] = columns
    return schema_info

schema_info = get_schema_info()
print("Database Schema:")
for table, columns in schema_info.items():
    print(f"Table: {table}")
    print(columns)

# Kiểm tra dữ liệu mẫu
print("Sample data:")
for table in db.get_usable_table_names():
    print(f"Table: {table}")
    print(db.run(f"SELECT * FROM {table} LIMIT 3"))

In [ ]:
import os
import getpass # Import the getpass module
os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass.getpass('Hugging Face Hub API Token:')

In [ ]:
# Khởi tạo Hugging Face LLM
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",  # Mô hình mạnh hơn
    model_kwargs={
        "temperature": 0.3,       # Giảm nhiệt độ để có kết quả nhất quán hơn
        "max_length": 1024,       # Tăng độ dài đầu ra
        "max_new_tokens": 512,    # Giới hạn tokens mới
        "top_p": 0.9              # Top p sampling
    }
)

In [ ]:
# Khởi tạo embedding model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [ ]:
# Các ví dụ để huấn luyện mô hình
examples = [
    {
        "input": "Danh mục sản phẩm có những gì?",
        "query": "SELECT id, name FROM categories"
    },
    {
        "input": "Liệt kê tất cả sản phẩm",
        "query": "SELECT id, name, price FROM products LIMIT 10"
    },
    {
        "input": "Danh sách sản phẩm có giá dưới 300 nghìn",
        "query": "SELECT name, price FROM products WHERE price < 300000 ORDER BY price DESC LIMIT 10"
    },
    {
        "input": "Những sản phẩm có giá từ 100 nghìn đến 500 nghìn",
        "query": "SELECT name, price FROM products WHERE price >= 100000 AND price <= 500000 ORDER BY price LIMIT 10"
    },
    {
        "input": "5 sản phẩm có giá cao nhất",
        "query": "SELECT name, price FROM products ORDER BY price DESC LIMIT 5"
    },
    {
        "input": "Sản phẩm thuộc danh mục nào",
        "query": "SELECT p.name AS product_name, c.name AS category_name FROM products p JOIN categories c ON p.category_id = c.id LIMIT 10"
    }
]

In [ ]:
# Khởi tạo example selector
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    Chroma,
    k=3,
    input_keys=['input']
)

In [ ]:
# Hàm định dạng kết quả SQL để hiển thị đẹp mắt hơn
def format_sql_result(result, query_type="default"):
    """
    Định dạng kết quả SQL thành bảng dễ đọc
    params:
        result: Kết quả từ việc chạy truy vấn SQL
        query_type: Loại truy vấn để định dạng phù hợp
    """
    try:
        # Xử lý kết quả dạng chuỗi
        if isinstance(result, str):
            # Kiểm tra nếu là kết quả rỗng
            if result.strip() == "":
                return "Không tìm thấy kết quả nào."

            # Thử chuyển đổi chuỗi thành cấu trúc dữ liệu
            try:
                data = ast.literal_eval(result)

                # Nếu là list rỗng
                if not data:
                    return "Không tìm thấy kết quả nào."

                # Nếu là list các tuple (thường là kết quả SQL)
                if isinstance(data, list) and all(isinstance(item, tuple) for item in data):
                    # Loại bỏ các bản ghi trùng lặp
                    unique_data = []
                    seen = set()
                    for item in data:
                        item_key = str(item)
                        if item_key not in seen:
                            seen.add(item_key)
                            unique_data.append(item)

                    # Tạo DataFrame từ dữ liệu đã được lọc trùng
                    if len(unique_data) > 0 and len(unique_data[0]) == 2:
                        # Trường hợp có 2 cột (thường là tên và giá)
                        df = pd.DataFrame(unique_data, columns=["Tên sản phẩm", "Giá (VNĐ)"])

                        # Định dạng cột giá
                        df["Giá (VNĐ)"] = df["Giá (VNĐ)"].apply(lambda x: f"{x:,.2f}" if isinstance(x, (int, float)) else x)

                        # Tạo bảng đẹp với tabulate
                        table = tabulate(df, headers="keys", tablefmt="grid", showindex=range(1, len(df) + 1))
                        return f"```\n{table}\n```"
                    else:
                        # Trường hợp số cột khác 2
                        headers = [f"Cột {i+1}" for i in range(len(unique_data[0]))] if unique_data else []
                        df = pd.DataFrame(unique_data, columns=headers)
                        table = tabulate(df, headers="keys", tablefmt="grid", showindex=range(1, len(df) + 1))
                        return f"```\n{table}\n```"
            except (SyntaxError, ValueError) as e:
                # Nếu không parse được, trả về kết quả gốc
                return f"```\n{result}\n```"

        # Trường hợp result đã là cấu trúc dữ liệu
        elif isinstance(result, list):
            if not result:
                return "Không tìm thấy kết quả nào."

            # Xử lý danh sách các tuple
            if all(isinstance(item, tuple) for item in result):
                # Loại bỏ các bản ghi trùng lặp
                unique_data = []
                seen = set()
                for item in result:
                    item_key = str(item)
                    if item_key not in seen:
                        seen.add(item_key)
                        unique_data.append(item)

                if len(unique_data) > 0:
                    if len(unique_data[0]) == 2:
                        df = pd.DataFrame(unique_data, columns=["Tên sản phẩm", "Giá (VNĐ)"])
                        df["Giá (VNĐ)"] = df["Giá (VNĐ)"].apply(lambda x: f"{x:,.2f}" if isinstance(x, (int, float)) else x)
                        table = tabulate(df, headers="keys", tablefmt="grid", showindex=range(1, len(df) + 1))
                        return f"```\n{table}\n```"
                    else:
                        headers = [f"Cột {i+1}" for i in range(len(unique_data[0]))]
                        df = pd.DataFrame(unique_data, columns=headers)
                        table = tabulate(df, headers="keys", tablefmt="grid", showindex=range(1, len(df) + 1))
                        return f"```\n{table}\n```"

            # Xử lý danh sách đơn giản
            return "\n".join([f"{i+1}. {item}" for i, item in enumerate(result)])

        # Fallback: trả về kết quả gốc
        return f"```\n{result}\n```"

    except Exception as e:
        print(f"Error formatting result: {e}")
        # Trường hợp lỗi, trả về chuỗi gốc
        return f"```\n{result}\n```"

In [ ]:
# Hàm truy vấn dữ liệu và chuyển kết quả về dạng list
def query_as_list(db, query):
    res = db.run(query)
    try:
        # Xử lý kết quả trả về dưới dạng chuỗi
        if isinstance(res, str):
            # Cố gắng parse chuỗi thành list
            try:
                res = ast.literal_eval(res)
                if isinstance(res, list):
                    # Làm phẳng danh sách nếu là danh sách lồng nhau
                    flat_res = []
                    for item in res:
                        if isinstance(item, (list, tuple)):
                            flat_res.extend([el for el in item if el])
                        else:
                            if item:
                                flat_res.append(item)
                    return list(set(flat_res))
            except (SyntaxError, ValueError):
                # Nếu không parse được, tách chuỗi theo dấu phẩy
                return [item.strip() for item in res.split(',') if item.strip()]
        elif isinstance(res, (list, tuple)):
            # Nếu đã là list hoặc tuple
            flat_res = []
            for item in res:
                if isinstance(item, (list, tuple)):
                    flat_res.extend([el for el in item if el])
                else:
                    if item:
                        flat_res.append(item)
            return list(set(flat_res))

        # Fallback: trả về list rỗng nếu không xử lý được
        return []
    except Exception as e:
        print(f"Error in query_as_list: {e}")
        return []

# Lấy danh sách tất cả tên sản phẩm và danh mục
try:
    products = query_as_list(db, "SELECT name FROM products")
    categories = query_as_list(db, "SELECT name FROM categories")
    all_items = products + categories

    # Xử lý thêm các thuộc tính quan trọng
    try:
        price_ranges = [
            "dưới 100 nghìn", "dưới 200 nghìn", "dưới 300 nghìn", "dưới 500 nghìn",
            "từ 100 nghìn đến 300 nghìn", "từ 300 nghìn đến 500 nghìn",
            "từ 500 nghìn đến 1 triệu", "trên 1 triệu"
        ]
        all_items.extend(price_ranges)
    except Exception as e:
        print(f"Error adding price ranges: {e}")

    print(f"Đã tải {len(all_items)} tên sản phẩm, danh mục và thuộc tính")
except Exception as e:
    print(f"Error loading product and category names: {e}")
    all_items = []

In [ ]:
# Tạo vectordb cho các tên sản phẩm và thuộc tính
vector_db = Chroma.from_texts(all_items, embeddings)
retriever = vector_db.as_retriever(search_kwargs={"k": 20})

In [ ]:
# Mô tả công cụ tìm kiếm
description = """
Công cụ tìm kiếm tên sản phẩm, danh mục và thuộc tính.
Đầu vào: Tên gần đúng hoặc một phần của sản phẩm/danh mục.
Đầu ra: Danh sách tên chính xác mà bạn có thể sử dụng trong câu truy vấn SQL.
Sử dụng tên chính xác trả về khi lọc trong SQL.
"""

In [ ]:
# Tạo retriever tool
retriever_tool = create_retriever_tool(
    retriever,
    description=description,
    name="search_products_and_categories"
)

In [ ]:
# Xử lý giá trị tiền tệ từ tiếng Việt
def extract_price_value(text):
    # Tìm tất cả các số trong văn bản
    numbers = re.findall(r'\d+', text)

    if not numbers:
        return None

    # Lấy số lớn nhất (giả định là giá trị tiền)
    price = int(max(numbers, key=int))

    # Xác định đơn vị tiền tệ
    if "triệu" in text or "tr" in text:
        price *= 1000000
    elif "nghìn" in text or "ngàn" in text or "k" in text:
        price *= 1000

    return price

In [ ]:
# Hàm phân tích loại truy vấn
def analyze_query_type(query_text):
    query_text = query_text.lower()

    if any(word in query_text for word in ["danh sách", "liệt kê", "hiển thị", "xem"]):
        # Xác định nếu là truy vấn về giá
        if any(word in query_text for word in ["giá", "price", "tiền"]):
            if any(word in query_text for word in ["dưới", "nhỏ hơn", "ít hơn", "không quá"]):
                return "price_less_than"
            elif any(word in query_text for word in ["trên", "lớn hơn", "nhiều hơn", "trên"]):
                return "price_greater_than"
            elif any(word in query_text for word in ["từ", "giữa", "trong khoảng"]):
                return "price_range"
            elif any(word in query_text for word in ["đắt nhất", "cao nhất"]):
                return "price_highest"
            elif any(word in query_text for word in ["rẻ nhất", "thấp nhất"]):
                return "price_lowest"
            return "price_query"

        # Xác định nếu là truy vấn về danh mục
        if any(word in query_text for word in ["danh mục", "category", "loại"]):
            return "category_list"

        # Xác định nếu là truy vấn về sản phẩm trong danh mục
        if "thuộc" in query_text or "trong danh mục" in query_text:
            return "products_in_category"

        # Mặc định là danh sách sản phẩm
        return "product_list"

    # Truy vấn về thông tin cụ thể
    if any(word in query_text for word in ["đắt nhất", "cao nhất"]):
        return "price_highest"
    if any(word in query_text for word in ["rẻ nhất", "thấp nhất"]):
        return "price_lowest"

    # Mặc định
    return "unknown"

In [ ]:
# Hàm xử lý truy vấn của người dùng
def get_answer(user_query):
    # Phân tích loại truy vấn
    query_type = analyze_query_type(user_query)
    print(f"Phân tích truy vấn: {query_type}")

    top_k = 20
    dialect = db.dialect

    # Xây dựng system prompt
    system_prefix = f"""
    Bạn là một agent trợ lý truy vấn SQL cho cơ sở dữ liệu bán hàng.
    Nhiệm vụ của bạn là chuyển đổi câu hỏi tiếng Việt thành truy vấn SQL hợp lệ, thực thi và trả về kết quả.

    QUAN TRỌNG: Khi làm việc với giá sản phẩm:
    - Giá được lưu bằng đơn vị cơ bản (VND) trong cơ sở dữ liệu, không phải đơn vị "nghìn" hay "triệu"
    - Khi người dùng nói "300 nghìn" bạn phải chuyển thành 300000 trong truy vấn SQL
    - Khi người dùng nói "1 triệu" bạn phải chuyển thành 1000000 trong truy vấn SQL

    Loại truy vấn hiện tại: {query_type}

    Tuân theo các quy tắc sau:
    1. Tạo một truy vấn {dialect} hợp lệ về mặt cú pháp
    2. Luôn giới hạn kết quả tối đa {top_k} bản ghi trừ khi người dùng yêu cầu số lượng cụ thể
    3. Bạn có thể sắp xếp kết quả để hiển thị các ví dụ thú vị nhất
    4. Chỉ truy vấn các cột liên quan, không lấy tất cả các cột
    5. Kiểm tra truy vấn của bạn trước khi thực thi, nếu gặp lỗi, viết lại và thử lại
    6. KHÔNG thực hiện bất kỳ câu lệnh DML nào (INSERT, UPDATE, DELETE, DROP, v.v.)

    Nếu câu hỏi không liên quan đến cơ sở dữ liệu, trả về "Tôi không biết" làm câu trả lời.
    """

    # Format ví dụ với câu truy vấn tương ứng
    example_prompt = PromptTemplate.from_template("Câu hỏi: {input}\nCâu truy vấn SQL: {query}")

    try:
        # Chọn các ví dụ tương tự với câu hỏi của người dùng
        safe_examples = example_selector.select_examples({"input": user_query})

        # Tạo few-shot prompt template
        few_shot_prompt = FewShotPromptTemplate(
            examples=safe_examples,
            example_prompt=example_prompt,
            prefix=system_prefix,
            suffix="",
            input_variables=["input"],
        )
    except Exception as e:
        print(f"Lỗi khi chọn ví dụ: {e}")
        # Sử dụng tất cả ví dụ nếu có lỗi
        few_shot_prompt = FewShotPromptTemplate(
            examples=examples,
            example_prompt=example_prompt,
            prefix=system_prefix,
            suffix="",
            input_variables=["input"],
        )

    # Format prompt với câu hỏi của người dùng
    prompt_val = few_shot_prompt.format(input=user_query)

    # Thêm thông tin về schema cơ sở dữ liệu
    schema_text = "Thông tin về schema của cơ sở dữ liệu:\n"
    for table, columns in schema_info.items():
        schema_text += f"Bảng {table}: {columns}\n"

    # Thêm hướng dẫn về việc tìm kiếm tên riêng
    system_unique_name_prompt = f"""
    Đây là thông tin bổ sung về cơ sở dữ liệu:
    {schema_text}

    Các bảng có sẵn: {', '.join(db.get_usable_table_names())}

    Nếu bạn cần lọc theo tên sản phẩm hoặc danh mục, luôn sử dụng công cụ "search_products_and_categories" trước để tìm giá trị chính xác!

    Để xử lý các truy vấn về giá:
    - Khi truy vấn "dưới X nghìn", sử dụng WHERE price < X*1000
    - Khi truy vấn "từ X nghìn đến Y nghìn", sử dụng WHERE price >= X*1000 AND price <= Y*1000
    - Khi truy vấn "trên X nghìn", sử dụng WHERE price > X*1000

    Trả về kết quả với thông tin chi tiết và định dạng đẹp.
    """

    final_prompt = prompt_val + "\n" + system_unique_name_prompt

    # Tạo prompt template cho agent
    full_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", final_prompt),
            ("human", "{input}"),
            MessagesPlaceholder("agent_scratchpad"),
        ]
    )

    # Tạo SQL agent
    try:
        agent = create_sql_agent(
            llm=llm,
            db=db,
            max_iterations=7,  # Tăng số lượng iterations
            extra_tools=[retriever_tool],
            prompt=full_prompt,
            agent_type="zero-shot-react-description",
            verbose=True
        )

        # Thực thi agent
        result = agent.invoke({"input": user_query})
        raw_output = result['output']

        # Xử lý đầu ra để trích xuất kết quả SQL nếu có
        sql_result_pattern = r"SELECT.*?FROM.*?(?=\n\n|\Z)"
        sql_matches = re.findall(sql_result_pattern, raw_output, re.DOTALL | re.IGNORECASE)

        # Tìm và định dạng kết quả SQL
        if "Kết quả tìm kiếm" in raw_output or "Danh sách" in raw_output:
            # Chia nhỏ để lấy phần kết quả
            parts = re.split(r"Kết quả tìm kiếm|Danh sách", raw_output, 1)
            if len(parts) > 1:
                intro = parts[0] + ("Kết quả tìm kiếm" if "Kết quả tìm kiếm" in raw_output else "Danh sách")
                result_part = parts[1]

                # Tìm dữ liệu có thể là kết quả SQL
                data_match = re.search(r"\[\(.*?\)\]", result_part, re.DOTALL)
                if data_match:
                    try:
                        data_str = data_match.group(0)
                        formatted_result = format_sql_result(data_str, query_type)
                        return intro + "\n" + formatted_result
                    except Exception as e:
                        print(f"Lỗi khi định dạng kết quả: {e}")

        # Nếu không tìm thấy mẫu kết quả SQL, trả về kết quả gốc
        return raw_output
    except Exception as e:
        print(f"Lỗi khi thực thi agent: {e}")

        # Fallback cải tiến: Xử lý các loại truy vấn phổ biến
        try:
            # Xử lý truy vấn về giá
            if query_type.startswith("price_"):
                if query_type == "price_less_than":
                    # Xử lý giá dưới X
                    price = extract_price_value(user_query)
                    if price:
                        query = f"SELECT name, price FROM products WHERE price < {price} ORDER BY price DESC LIMIT 10"
                        raw_result = db.run(query)
                        formatted_result = format_sql_result(raw_result, "price_less_than")
                        return f"Danh sách sản phẩm có giá dưới {price:,} đồng:\n{formatted_result}"

                elif query_type == "price_range":
                    # Xử lý giá trong khoảng
                    numbers = re.findall(r'\d+', user_query)
                    if len(numbers) >= 2:
                        # Xác định đơn vị tiền tệ
                        if "triệu" in user_query:
                            min_price = int(numbers[0]) * 1000000
                            max_price = int(numbers[1]) * 1000000
                        else:  # Mặc định là nghìn
                            min_price = int(numbers[0]) * 1000
                            max_price = int(numbers[1]) * 1000

                        query = f"SELECT name, price FROM products WHERE price >= {min_price} AND price <= {max_price} ORDER BY price LIMIT 10"
                        raw_result = db.run(query)
                        formatted_result = format_sql_result(raw_result, "price_range")
                        return f"Danh sách sản phẩm có giá từ {min_price:,} đến {max_price:,} đồng:\n{formatted_result}"

                elif query_type == "price_highest":
                    # Xử lý sản phẩm có giá cao nhất
                    match = re.search(r'(\d+)', user_query)
                    limit = int(match.group(1)) if match else 5

                    query = f"SELECT name, price FROM products ORDER BY price DESC LIMIT {limit}"
                    raw_result = db.run(query)
                    formatted_result = format_sql_result(raw_result, "price_highest")
                    return f"{limit} sản phẩm có giá cao nhất:\n{formatted_result}"

                elif query_type == "price_lowest":
                    # Xử lý sản phẩm có giá thấp nhất
                    match = re.search(r'(\d+)', user_query)
                    limit = int(match.group(1)) if match else 5

                    query = f"SELECT name, price FROM products ORDER BY price ASC LIMIT {limit}"
                    raw_result = db.run(query)
                    formatted_result = format_sql_result(raw_result, "price_lowest")
                    return f"{limit} sản phẩm có giá thấp nhất:\n{formatted_result}"

                elif query_type == "price_query":
                    # Truy vấn giá chung về sản phẩm
                    # Tìm tên sản phẩm
                    related_products = retriever.get_relevant_documents(user_query)
                    if related_products:
                        product_name = related_products[0].page_content
                        query = f"SELECT name, price FROM products WHERE name LIKE '%{product_name.split()[0]}%' LIMIT 5"
                        raw_result = db.run(query)
                        formatted_result = format_sql_result(raw_result, "price_query")
                        return f"Thông tin giá của sản phẩm '{product_name}':\n{formatted_result}"

            # Xử lý truy vấn danh mục
            elif any(word in user_query.lower() for word in ["danh mục", "category", "loại"]):
                if "sản phẩm trong" in user_query.lower() or "thuộc" in user_query.lower():
                    # Tìm sản phẩm trong danh mục
                    related_categories = retriever.get_relevant_documents(user_query)
                    if related_categories:
                        category_name = related_categories[0].page_content
                        query = f"""
                        SELECT p.name, p.price
                        FROM products p
                        JOIN categories c ON p.category_id = c.id
                        WHERE c.name LIKE '%{category_name}%'
                        LIMIT 10
                        """
                        result = db.run(query)
                        return f"Sản phẩm thuộc danh mục '{category_name}':\n{result}"
                else:
                    # Liệt kê danh mục
                    query = "SELECT id, name FROM categories"
                    result = db.run(query)
                    return f"Danh sách các danh mục:\n{result}"

            # Xử lý liệt kê tất cả sản phẩm
            elif any(phrase in user_query.lower() for phrase in ["tất cả sản phẩm", "danh sách sản phẩm", "liệt kê sản phẩm"]):
                query = "SELECT name, price FROM products LIMIT 10"
                result = db.run(query)
                return f"Danh sách sản phẩm:\n{result}"

            # Fallback cuối cùng
            return "Tôi không thể xử lý yêu cầu này. Vui lòng thử với câu hỏi cụ thể hơn về sản phẩm, danh mục hoặc giá cả."
        except Exception as fallback_error:
            print(f"Lỗi trong fallback: {fallback_error}")
            return "Xin lỗi, tôi gặp lỗi khi xử lý yêu cầu của bạn. Vui lòng thử lại với câu hỏi đơn giản hơn."

In [ ]:
# Hàm xử lý cuộc hội thoại
def chat(user_message, history):
    if history is None:
        history = []

    # Lưu lịch sử hội thoại
    message_history = history.copy()
    message_history.append({"role": "user", "content": user_message})

    # Đơn giản hóa đầu vào nếu lịch sử quá dài
    if len(message_history) > 6:  # Giữ 3 cặp hội thoại gần nhất
        context = "Dựa trên lịch sử hội thoại gần đây, hãy trả lời câu hỏi: " + user_message
    else:
        context = user_message

    # Thực hiện truy vấn
    try:
        result = get_answer(context)
        bot_reply = result
    except Exception as e:
        print(f"Lỗi không xác định: {e}")
        bot_reply = "Xin lỗi, tôi gặp lỗi khi xử lý yêu cầu của bạn. Vui lòng thử lại sau."

    # Cập nhật lịch sử
    history.append({"role": "user", "content": user_message})
    history.append({"role": "assistant", "content": bot_reply})

    return "", history

In [ ]:
# Tạo giao diện người dùng
with gr.Blocks() as demo:
    gr.Markdown("# 🤖 Chatbot SQL với Mistral 7B")
    gr.Markdown("""
    Chatbot này sử dụng mô hình Mistral 7B để truy vấn cơ sở dữ liệu sản phẩm.
    Bạn có thể hỏi về danh mục sản phẩm, giá cả, và nhiều thông tin khác.

    **Ví dụ câu hỏi:**
    - Danh mục sản phẩm có những gì?
    - Danh sách sản phẩm có giá dưới 300k
    - 5 sản phẩm có giá cao nhất
    - Sản phẩm thuộc danh mục nào
    """)

    chatbot = gr.Chatbot(
        type="messages",
        height=500,
        show_label=False,
        elem_id="chatbot"
    )

    with gr.Row():
        txt = gr.Textbox(
            show_label=False,
            placeholder="Nhập câu hỏi của bạn...",
            container=False,
            scale=9
        )
        submit_btn = gr.Button("Gửi", scale=1)

    clear_btn = gr.Button("Xóa lịch sử")

    txt.submit(chat, [txt, chatbot], [txt, chatbot])
    submit_btn.click(chat, [txt, chatbot], [txt, chatbot])
    clear_btn.click(lambda: (None, None), None, [txt, chatbot])

In [ ]:
# Khởi chạy ứng dụng
if __name__ == "__main__":
    demo.launch(share=True)